In [ ]:
from langchain.document_loaders import pyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Extract Data from the PDf File


def load_pdf_file(data):
    loader= DirectoryLoader(data, glob= "*.pdf", loader_cls= PyPDFLoader)

    documents = loader.load()

    return documents

In [ ]:
extracted_data= load_pdf_file(data= 'Data/')

In [ ]:
#extracted_data

In [ ]:
#Split the data into Text Chunks
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 20)
    text_chunks= text_splitter_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks= text_split(extracted_data)
# print('Length of Text Chunks', len(text_chunks))

In [ ]:
# text_chunks

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# Download the Embedding from Hugging Face
def download_hugging_face_embeddings():
    HuggingFaceEmbeddings(model_name= 'sentece-transforms/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
query_result= embeddings.embed_query('Heloow World')
# print('Length', len(query_result))  # give dimension of huggingmodel 

In [ ]:
# query_result

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY= os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

In [ ]:
pc= Pinecone(api_key=PINECONE_API_KEY)
index_name= 'medicalbot'

pc.create_index(
    name= index_name,
    dimentsion= 384,
    metric='cosine',
    spec=ServerlessSpace(
        cloud='aws',
        region='us-east-1'
    )
)

In [ ]:
import os
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
os.environ['OPENAI_API_KEY']= OPENAI_API_KEY

In [ ]:
from langchain_pincone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks, 
    index_name= index_name,
    embedding=embeddings,
)

In [ ]:
# Existing Index 

from langchain_pinecone import PineconeVecoreStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectoreStore.from_existing_index(
    index_name= index_name
    embedding= embeddings)


In [ ]:
# docsearch

In [ ]:
retriever= docsearch.as_retriever(search_type= 'similarity', search_kwargs= {"k": 3})
retrieved_docs= retriever.invoke("What is Acne , How to get medicine?")

In [ ]:
# print(retrieved_docs)

In [ ]:
from langchain_openai import OpenAI
llm= OpenAI(temperature= 0.5, max_tokens= 500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chain.combine_documents import create_stuff_documents_chain
from langchain.core.prompts import ChatPromptTemplete




system_prompt= (
    "You are an assistant for question-answering tasks. "
    "Use the following piece of the retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't Know . Use three sentences maximum and keep the "
    "answer concise."
    "{context}"
)



prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{inpute}"),
        
    ]
)

In [ ]:
question_answer_chain =create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response= rag_chain.invoke(['input': 'What is Acne? And What is gigantism?'])
# print(response['answer'])